In [17]:
!pip install -U openai-whisper flask flask_cors pyngrok
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [18]:
!ngrok config add-authtoken 2vtEFj0tGUjOXWdKpEzRw0lm9hI_6aATradqDJhfp3iB9SjK1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [20]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import whisper
import os
import uuid

In [21]:
model = whisper.load_model("base")

In [22]:
app = Flask(__name__)
CORS(app)

In [23]:
@app.route('/stten', methods=['POST'])
def transcribe_audio():
    if 'audio' not in request.files:
        return jsonify({'error': 'No audio file provided'}), 400

    audio_file = request.files['audio']
    filename = f"/content/{uuid.uuid4()}.wav"
    audio_file.save(filename)

    # Transcribe using Whisper
    try:
        result = model.transcribe(filename)
        text = result["text"]
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        os.remove(filename)

    return jsonify({'text': text})


In [25]:
public_url = ngrok.connect(5000)
print(f"🔥 Public URL: {public_url}")

app.run(port=5000)

🔥 Public URL: NgrokTunnel: "https://bbb9-34-143-246-70.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 13:50:14] "POST /stten HTTP/1.1" 200 -
